In [39]:
import requests
from bs4 import BeautifulSoup

# Prepare CSV file
topRatedCSV = "topRatedMovies.csv"
headers = "tmdb_id\ttitle\trating\n"
f = open(topRatedCSV, "w+")
f.write(headers)

# Scrape function
def scrapePage(headers, url, n_pages):
    # Loop through each page
    for page in range(1, n_pages+1):
        res_topRated = requests.get(url+'?page='+str(page), headers=headers)
        topRated_html = BeautifulSoup(res_topRated.content, "html.parser")

        print(url+'?page='+str(page), res_topRated.status_code, "OK")

        topRated_container = topRated_html.find("div", {"class": "media_items"})
        topRated_titles = topRated_container.find_all('a', {"class": "image"}) # type: ignore
        topRated_ratings = topRated_container.find_all('div', {"class": "user_score_chart"}) # type: ignore

        for title in topRated_titles:
            movie_id = topRated_titles[topRated_titles.index(title)].get('href').split('/')[2]
            movie_title = topRated_titles[topRated_titles.index(title)].get('title')
            movie_rating = topRated_ratings[topRated_titles.index(title)].get('data-percent').split('.')[0]
            print(movie_id, movie_title, movie_rating, sep='\t', file=f)

In [40]:
# Scrape
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13.3; rv:109) Gecko/20100101 Firefox/112.0'}
url = 'https://www.themoviedb.org/movie/top-rated'
n_pages = 5
scrapePage(headers, url, n_pages)

# Close CSV file
f.close()

https://www.themoviedb.org/movie/top-rated?page=1 200 OK
https://www.themoviedb.org/movie/top-rated?page=2 200 OK
https://www.themoviedb.org/movie/top-rated?page=3 200 OK
https://www.themoviedb.org/movie/top-rated?page=4 200 OK
https://www.themoviedb.org/movie/top-rated?page=5 200 OK
